In [2]:
# http://pandas.pydata.org/pandas-docs/version/0.15.2/10min.html
import pandas as pd
import os
import numpy as np
import scipy
import math
import matplotlib.pyplot as plt
from sklearn.covariance import GraphLassoCV
from sklearn.metrics import mutual_info_score
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy import stats
import matplotlib.patches as patches
import scipy.io
import random

In [3]:
labels_Glasser = pd.read_csv("labels_Glasser.csv",header=None)[0].tolist()
Nvars = len(labels_Glasser)

In [143]:
# give a path to the folder with the resting state datasets, day 1:
condition = 'restingstate_day2'  # restingstate_day1 / restingstate_day2 / WM / MOTOR
if condition == 'restingstate_day1':
    T = 2389
if condition == 'restingstate_day2':
    T = 2389
if condition == 'WM':    
    T = 799
if condition == 'MOTOR':
    T = 557

path = '/Users/nataliabielczyk/Desktop/Brainhack/PIPELINE/datasets/HCP_parcellated_smoothing5_100subjects_Glasser_' + condition + '/'
list = os.listdir(path) # dir is your directory path
number_files = len(list)

Nsubjects = 0
indexes_subjects = []
for ind in range(number_files):
    filename = path + list[ind]
    if filename[-3:] == 'csv':
        Nsubjects = Nsubjects + 1
        indexes_subjects.append(ind)

In [85]:
# [01] Pearson's r:
# compute Pearson's r for all the subjects:
pearsonr_all = np.zeros((Nsubjects,Nvars,Nvars))
k = 0
for ind in range(number_files):
    filename = path + list[ind]
    if filename[-3:] == 'csv':
        
        df1    = pd.read_csv(filename)
        data1  = df1.values
        Nvars  = data1.shape[1] 
        
        ## shorten the datasets to 2 X 400, or 2 x 279 samples, in order to make them comparable with the visual stimulation task:
        #Nshort = 400
        #list1 = np.ndarray.tolist(np.arange(Nshort))
        #list2 = np.ndarray.tolist(1200+np.arange(Nshort))
        #indexes = list1 + list2       
        #data1 = data1[indexes,:]
        
        pearsonr = np.zeros((Nvars,Nvars))
        pearsonr_p = np.zeros((Nvars,Nvars))
        for ind1 in range(Nvars):
            for ind2 in range(Nvars):
                (pearsonr[ind1,ind2], pearsonr_p[ind1,ind2]) = scipy.stats.pearsonr(data1[:,ind1], data1[:,ind2])
        
        pearsonr_all[k,:,:] = pearsonr
        k = k + 1   
        print k

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104


In [86]:
foldername = 'output_second_level/'
if not os.path.exists(foldername):
    os.makedirs(foldername)
    
filename = foldername + 'pearsonr_all_' + condition + '.mat'
scipy.io.savemat(filename, {'pearsonr_all': pearsonr_all})

In [81]:
# surrogate version of Pearson's r:
pearsonr_all_surrogate = np.zeros((Nsubjects,Nvars,Nvars))

for ind in range(Nsubjects):
    print ind
    # combine a surrogate subject from multiple labels:
    surrogate_subject = np.zeros((T,Nvars))
    # choose time series for every ROI from random subjects:
    for ind1 in range(Nvars):
        ind2                      = random.sample(range(1, Nsubjects), 1)[0] # random subject number
        filename                  = path + list[indexes_subjects[ind2]]
        df1                       = pd.read_csv(filename)
        data1                     = df1.values 
        surrogate_subject[:,ind1] = data1[:,ind1]
        
    # shorten the datasets to 2 X 400, or 2 x 279 samples, in order to make them comparable with the visual stimulation task:
    #Nshort = 400
    #list1 = np.ndarray.tolist(np.arange(Nshort))
    #list2 = np.ndarray.tolist(1200+np.arange(Nshort))
    #indexes = list1 + list2       
    #data1 = data1[indexes,:]
        
    pearsonr = np.zeros((Nvars,Nvars))
    pearsonr_p = np.zeros((Nvars,Nvars))
    for ind1 in range(Nvars):
        for ind2 in range(Nvars):
            (pearsonr[ind1,ind2], pearsonr_p[ind1,ind2]) = scipy.stats.pearsonr(data1[:,ind1], data1[:,ind2])
      
    pearsonr_all_surrogate[ind,:,:] = pearsonr  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


In [117]:
foldername = 'output_second_level/'
if not os.path.exists(foldername):
    os.makedirs(foldername)
    
filename = foldername + 'pearsonr_all_surrogate_' + condition + '.mat'
scipy.io.savemat(filename, {'pearsonr_all_surrogate': pearsonr_all_surrogate})

In [139]:
# [02] partial correlation:
# compute partial correlation for all the subjects:
partial_correlation_all = np.zeros((Nsubjects,Nvars,Nvars))
k = 0
for ind in range(number_files):
    filename = path + list[ind]
    if filename[-3:] == 'csv':
        
        df1    = pd.read_csv(filename)
        data1  = df1.values
        Nvars  = data1.shape[1]  
        
        # shorten the datasets to 2 X 400, or 2 x 279 samples, in order to make them comparable with the visual stimulation task:
        Nshort = 400
        list1 = np.ndarray.tolist(np.arange(Nshort))
        list2 = np.ndarray.tolist(1200+np.arange(Nshort))
        indexes = list1 + list2       
        data1 = data1[indexes,:]
        
        covariance_matrix = np.cov(np.transpose(data1))
        inv_cov = np.linalg.inv(covariance_matrix)
        partial_correlation = np.zeros((Nvars,Nvars))
        for ind1 in range(Nvars):
            for ind2 in range(Nvars):
                partial_correlation[ind1,ind2] = - inv_cov[ind1,ind2]/np.sqrt((inv_cov[ind1,ind1])*(inv_cov[ind2,ind2]))
        
        partial_correlation_all[k,:,:] = partial_correlation
        k = k + 1  
        print k

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


KeyboardInterrupt: 

In [111]:
foldername = 'output_second_level/'
if not os.path.exists(foldername):
    os.makedirs(foldername)
    
filename = foldername + 'partial_correlation_all_' + condition + '_800samples.mat'
scipy.io.savemat(filename, {'partial_correlation_all': partial_correlation_all})

In [144]:
# surrogate version of partial correlation:
partial_correlation_all_surrogate = np.zeros((Nsubjects,Nvars,Nvars))

for ind in range(Nsubjects):
    print ind
    # combine a surrogate subject from multiple labels:
    surrogate_subject = np.zeros((T,Nvars))
    # choose time series for every ROI from random subjects:
    for ind1 in range(Nvars):
        ind2                      = random.sample(range(1, Nsubjects), 1)[0] # random subject number
        filename                  = path + list[indexes_subjects[ind2]]
        df1                       = pd.read_csv(filename)
        data1                     = df1.values 
        surrogate_subject[:,ind1] = data1[:,ind1]
        
    # shorten the datasets to 2 X 400, or 2 x 279 samples, in order to make them comparable with the visual stimulation task:
    Nshort = 279
    list1 = np.ndarray.tolist(np.arange(Nshort))
    list2 = np.ndarray.tolist(1200+np.arange(Nshort))
    indexes = list1 + list2       
    data1 = data1[indexes,:]
        
    covariance_matrix = np.cov(np.transpose(data1))
    inv_cov = np.linalg.inv(covariance_matrix)
    partial_correlation = np.zeros((Nvars,Nvars))
    for ind1 in range(Nvars):
        for ind2 in range(Nvars):
            partial_correlation[ind1,ind2] = - inv_cov[ind1,ind2]/np.sqrt((inv_cov[ind1,ind1])*(inv_cov[ind2,ind2]))
        
    partial_correlation_all_surrogate[ind,:,:] = partial_correlation

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


In [145]:
foldername = 'output_second_level/'
if not os.path.exists(foldername):
    os.makedirs(foldername)
    
filename = foldername + 'partial_correlation_all_surrogate_' + condition + '_558samples.mat'
scipy.io.savemat(filename, {'partial_correlation_all_surrogate': partial_correlation_all_surrogate})

In [14]:
# [04] mutual information:
# compute mutual information for all the subjects:
from sklearn.metrics import mutual_info_score
def calc_MI(X,Y,bins):
   c_XY = np.histogram2d(X,Y,bins)[0]
   c_X = np.histogram(X,bins)[0]
   c_Y = np.histogram(Y,bins)[0]

   H_X = shan_entropy(c_X)
   H_Y = shan_entropy(c_Y)
   H_XY = shan_entropy(c_XY)

   MI = H_X + H_Y - H_XY
   return MI
def shan_entropy(c):
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))  
    return H

bins = 10
mutual_information_all = np.zeros((Nsubjects,Nvars,Nvars))
k = 0
for ind in range(number_files):
    filename = path + list[ind]
    if filename[-3:] =='csv':
        
        df1    = pd.read_csv(filename)
        data1  = df1.values        
        Nvars  = data1.shape[1]  
        
        # shorten the datasets to 2 X 400, or 2 x 279 samples, in order to make them comparable with the visual stimulation task:
        Nshort = 400
        list1 = np.ndarray.tolist(np.arange(Nshort))
        list2 = np.ndarray.tolist(1200+np.arange(Nshort))
        indexes = list1 + list2       
        data1 = data1[indexes,:]
        
        matMI = np.zeros((Nvars,Nvars))

        for ix in np.arange(Nvars):
            for jx in np.arange(ix+1,Nvars):
                matMI[ix,jx] = calc_MI(data1[:,ix], data1[:,jx], bins)
                matMI[jx,ix] = matMI[ix,jx]
        
        mutual_information_all[k,:,:] = matMI
        k = k + 1  
        print k

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102


In [123]:
foldername = 'output_second_level/'
if not os.path.exists(foldername):
    os.makedirs(foldername)
    
filename = foldername + 'mutual_information_all_' + condition + '_800samples.mat'
scipy.io.savemat(filename, {'mutual_information_all': mutual_information_all})

In [27]:
# [04b] permutation testing for mutual information:
# create surrogate data:
mutual_information_all_surrogate = np.zeros((mutual_information_all.shape))

# compute mutual information for all the subjects:
from sklearn.metrics import mutual_info_score
def calc_MI(X,Y,bins):
   c_XY = np.histogram2d(X,Y,bins)[0]
   c_X = np.histogram(X,bins)[0]
   c_Y = np.histogram(Y,bins)[0]

   H_X = shan_entropy(c_X)
   H_Y = shan_entropy(c_Y)
   H_XY = shan_entropy(c_XY)

   MI = H_X + H_Y - H_XY
   return MI

def shan_entropy(c):
    c_normalized = c / float(np.sum(c))
    c_normalized = c_normalized[np.nonzero(c_normalized)]
    H = -sum(c_normalized* np.log2(c_normalized))  
    return H

bins = 10
for ind in range(Nsubjects):
    print ind
    # combine a surrogate subject from multiple labels:
    surrogate_subject = np.zeros((T,Nvars))
    # choose time series for every ROI from random subjects:
    for ind1 in range(Nvars):
        ind2                      = random.sample(range(1, Nsubjects), 1)[0] # random subject number
        filename                  = path + list[indexes_subjects[ind2]]
        df1                       = pd.read_csv(filename)
        data1                     = df1.values 
        #print filename
        #print data1.shape
        #print surrogate_subject.shape
        surrogate_subject[:,ind1] = data1[:,ind1]
        
    # shorten the datasets to 2 X 400, or 2 x 279 samples, in order to make them comparable with the visual stimulation task:
    Nshort = 279
    list1 = np.ndarray.tolist(np.arange(Nshort))
    list2 = np.ndarray.tolist(1200+np.arange(Nshort))
    indexes = list1 + list2       
    data1 = data1[indexes,:]
        
    matMI = np.zeros((Nvars,Nvars))

    for ix in np.arange(Nvars):
        for jx in np.arange(ix+1,Nvars):
            matMI[ix,jx] = calc_MI(surrogate_subject[:,ix], surrogate_subject[:,jx], bins)
            matMI[jx,ix] = matMI[ix,jx]    
    mutual_information_all_surrogate[ind,:,:] = matMI  

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101


In [28]:
foldername = 'output_second_level/'
if not os.path.exists(foldername):
    os.makedirs(foldername)
    
filename = foldername + 'mutual_information_all_surrogate_' + condition + '_800samples.mat'
scipy.io.savemat(filename, {'mutual_information_all_surrogate': mutual_information_all_surrogate})